In [110]:
import sys
print(sys.executable)

/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [111]:
import torch
import sys
import os

print("Python:", sys.executable)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")
print("Realpath:", os.path.realpath(sys.executable))
print(torch.__file__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

device = torch.device("cuda:1")





Python: /workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python
CUDA available: True
GPU name: NVIDIA RTX A6000
Realpath: /opt/conda/bin/python3.11
/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/lib/python3.11/site-packages/torch/__init__.py
12.4
90100


In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [113]:
import sys
print(sys.executable)

import os
os.environ["TRANSFORMERS_NO_TF"] = "1"




/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [114]:
import os

import pprint
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


from langchain_core.documents import Document


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [115]:
df = pd.read_csv("/workspace/miluser/Mitsuharu_2002/csv_data/TPPHunter_T_Again.csv")
df2 = pd.read_csv("/workspace/miluser/Mitsuharu_2002/csv_data/MITRE_ATTACKv15-1_TTP_Dataset.csv")

In [116]:
!nvidia-smi


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Dec 24 12:27:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:03:00.0 Off |                  Off |
| 30%   38C    P8             18W /  300W |      28MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [117]:
df

,ID,Domain,act_Technique ID,act_TacticsName,act_Technique Name,Sentences,test_or_train,Predicted_ATT&CK ID_original,Predicted_TacticsName
0,1,Enterprise,T1543,Persistence:Privilege Escalation,Create or Modify System Process: Windows Service,Carbanak malware installs itself as a service ...,train,NaN,NaN
1,2,Enterprise,T1562,Defense Evasion,Impair Defenses: Disable or Modify System Fire...,Carbanak may use netsh to add local firewall r...,train,NaN,NaN
2,3,Enterprise,T1036,Defense Evasion,Masquerading: Masquerade Task or Service,Carbanak has copied legitimate service names t...,train,NaN,NaN
3,4,Enterprise,T1036,Defense Evasion,Masquerading: Match Legitimate Name or Location,"Carbanak has named malware ""svchost.exe,"" whic...",train,NaN,NaN
4,5,Enterprise,T1588,Resource Development,Obtain Capabilities: Tool,Carbanak has obtained and used open-source too...,train,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8382,8383,Enterprise,T1573,Command and Control,Encrypted Channel: Symmetric Cryptography,NETEAGLE will decrypt resources it downloads w...,train,NaN,NaN
8383,8384,Enterprise,T1041,Exfiltration,Exfiltration Over C2 Channel,NETEAGLE is capable of reading files over the ...,train,NaN,NaN
8384,8385,Enterprise,T1083,Discovery,File and Directory Discovery,NETEAGLE allows adversaries to enumerate and m...,train,NaN,NaN
8385,8386,Enterprise,T1057,Discovery,Process Discovery,NETEAGLE can send process listings over the C2...,test,T1057,Exfiltration


In [118]:
df2

,Key,Value
0,T1094,Some Software variants use raw TCP for C2.
1,T1047,Software leverages WMI to enumerate anti-virus...
2,T1003.001,Software contains an implementation of Softwar...
3,T1614.001,Attacker has used implants to collect the syst...
4,T1587.001,For Software modular backdoor.
...,...,...
19742,T1045,Most decompression techniques decompress the e...
19743,T1059.001,Software uses PowerShell to download and execu...
19744,T1059.003,Software can use `cmd.exe` to execute a create...
19745,T1112,Software has functionality to remove Registry ...


In [119]:
len(set(df["act_Technique ID"]))
print(df.columns.tolist())


['ID', 'Domain', 'act_Technique ID', 'act_TacticsName', 'act_Technique Name', 'Sentences', 'test_or_train', 'Predicted_ATT&CK ID_original', 'Predicted_TacticsName']


In [120]:
df = df[["ID", "test_or_train", "act_Technique ID", "act_Technique Name","act_TacticsName", "Sentences", "Predicted_TacticsName"]]

In [121]:
# train, test = train_test_split(df, test_size=0.2, random_state=0)
import pandas as pd
import numpy as np

# test_or_train が "train" の行だけ
train = df2

# test_or_train が "test" の行だけ
test = df[df["test_or_train"] == "test"].copy()


In [122]:
print(len(test))
print(len(train))

1678
19747


In [123]:
test

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
14,15,test,T1005,Data from Local System,Collection,"Frankenstein has enumerated hosts via Empire, ...",Discovery
20,21,test,T1588,Obtain Capabilities: Tool,Resource Development,Frankenstein has obtained and used Empire to d...,Execution
22,23,test,T1566,Phishing: Spearphishing Attachment,Initial Access,Frankenstein has used spearphishing emails to ...,Initial Access
23,24,test,T1057,Process Discovery,Discovery,"Frankenstein has enumerated hosts, looking to ...",Discovery
29,30,test,T1204,User Execution: Malicious File,Execution,Frankenstein has used trojanized Microsoft Wor...,Execution
...,...,...,...,...,...,...,...
8368,8369,test,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,TYPEFRAME can open the Windows Firewall on the...,Defense Evasion
8369,8370,test,T1070,Indicator Removal on Host: File Deletion,Defense Evasion,TYPEFRAME can delete files off the system.[1],Defense Evasion
8381,8382,test,T1059,Command and Scripting Interpreter: Windows Com...,Execution,NETEAGLE allows adversaries to execute shell c...,Execution
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Exfiltration


In [124]:
train

,Key,Value
0,T1094,Some Software variants use raw TCP for C2.
1,T1047,Software leverages WMI to enumerate anti-virus...
2,T1003.001,Software contains an implementation of Softwar...
3,T1614.001,Attacker has used implants to collect the syst...
4,T1587.001,For Software modular backdoor.
...,...,...
19742,T1045,Most decompression techniques decompress the e...
19743,T1059.001,Software uses PowerShell to download and execu...
19744,T1059.003,Software can use `cmd.exe` to execute a create...
19745,T1112,Software has functionality to remove Registry ...


In [125]:
import pandas as pd
import re

def base_tid(x: str) -> str | None:
    """T1587.001 -> T1587 / 'aaa T1587.001 bbb' -> T1587 / None -> None"""
    if pd.isna(x):
        return None
    s = str(x)
    m = re.search(r"(T\d{4})", s)   # まず親IDだけ拾う
    return m.group(1) if m else None

# train: columns = ["Key", "Value"]
# test : columns = ["act_Technique ID", "Sentences"]

# 1) testに出てくる親ID集合を作る
test_base_set = set(
    test["act_Technique ID"]
      .map(base_tid)
      .dropna()
      .unique()
)

# 2) train側も親IDを作って、含まれるものだけ残す
train2 = train.copy()
train2["_base"] = train2["Key"].map(base_tid)

train_filtered = train2[train2["_base"].isin(test_base_set)].drop(columns=["_base"])

print("before:", len(train), "after:", len(train_filtered))


before: 19747 after: 12660


In [126]:
# Key に "." が含まれる = サブテクニック とみなして除外
train_main = train[~train["Key"].astype(str).str.contains(r"\.", na=False)].reset_index(drop=True)


In [127]:
train_main

,Key,Value
0,T1094,Some Software variants use raw TCP for C2.
1,T1047,Software leverages WMI to enumerate anti-virus...
2,T1559,Software has the ability to move data between ...
3,T1518,Software can list PHP server configuration det...
4,T1134,Software has used <code>SeDebugPrivilege</code...
...,...,...
8842,T1033,"During , threat actors enumerated sessions and..."
8843,T1057,Software can enumerate the current process on ...
8844,T1038,Adversaries may also directly modify the way a...
8845,T1045,Most decompression techniques decompress the e...


In [128]:
import pandas as pd
import re

def base_tid(x):
    if pd.isna(x):
        return None
    m = re.search(r"(T\d{4})", str(x))
    return m.group(1) if m else None

# train: ["Key","Value"]  (TechniqueIDはKeyに入ってる想定)
# test : ["act_Technique ID","Sentences"]

train2 = train.copy()
test2  = test.copy()

train2["_base"] = train2["Key"].map(base_tid)
test2["_base"]  = test2["act_Technique ID"].map(base_tid)

train_base_set = set(train2["_base"].dropna().unique())
test_base_set  = set(test2["_base"].dropna().unique())

missing_in_train = sorted(test_base_set - train_base_set)  # testにいるのにtrainにいない親ID
extra_in_train   = sorted(train_base_set - test_base_set)  # trainにいるのにtestにいない親ID（本来0のはず）

print("test parent IDs:", len(test_base_set))
print("train parent IDs:", len(train_base_set))
print("missing_in_train:", len(missing_in_train))
print("extra_in_train:", len(extra_in_train))

# 中身も見たい場合
print("missing_in_train examples:", missing_in_train[:30])
print("extra_in_train examples:", extra_in_train[:30])


test parent IDs: 50
train parent IDs: 342
missing_in_train: 0
extra_in_train: 292
missing_in_train examples: []
extra_in_train examples: ['T1001', 'T1002', 'T1004', 'T1006', 'T1007', 'T1008', 'T1009', 'T1010', 'T1011', 'T1013', 'T1014', 'T1015', 'T1017', 'T1019', 'T1020', 'T1022', 'T1023', 'T1024', 'T1025', 'T1026', 'T1028', 'T1029', 'T1030', 'T1031', 'T1032', 'T1034', 'T1035', 'T1037', 'T1038', 'T1039']


In [129]:
# train_filtered を使ってもう一回差分確認
trainF = train_filtered.copy()
trainF["_base"] = trainF["Key"].map(base_tid)

trainF_base_set = set(trainF["_base"].dropna().unique())

missing_in_trainF = sorted(test_base_set - trainF_base_set)
extra_in_trainF   = sorted(trainF_base_set - test_base_set)

print("train_filtered parent IDs:", len(trainF_base_set))
print("missing_in_trainF:", len(missing_in_trainF))
print("extra_in_trainF:", len(extra_in_trainF))
print("missing_in_trainF examples:", missing_in_trainF[:30])
print("extra_in_trainF examples:", extra_in_trainF[:30])


train_filtered parent IDs: 50
missing_in_trainF: 0
extra_in_trainF: 0
missing_in_trainF examples: []
extra_in_trainF examples: []


In [130]:
train = train_main

In [131]:
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

device = "cuda:1"
model = SentenceTransformer(
    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1", #GPU1を使用
    device=device
)
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
# model = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

docs = []
texts = []
embeddings_list = []

for index, row in train.iterrows(): #dictionaly_Mitsu
    text = row["Value"]
    emb = model.encode(text, convert_to_numpy=True)

    doc = Document(
        page_content=text,
        metadata={
            "TTP_ID": row["Key"]        # ATT&CK IDの正解ラベル（T1059とか）
        },
    )

    docs.append(doc)
    texts.append(text)
    embeddings_list.append(emb)



In [132]:
docs[0]

Document(metadata={'TTP_ID': 'T1094'}, page_content='Some Software variants use raw TCP for C2.')

In [133]:
import faiss
import numpy as np

# ① ベクトル次元
d = embeddings_list[0].shape[0]  # 384 次元とか

# ② FAISS index を作成（L2距離）
index = faiss.IndexFlatL2(d)

# ③ numpy 配列にまとめる
emb_matrix = np.vstack(embeddings_list).astype('float32')

# ④ インデックスへ追加
index.add(emb_matrix)

print("FAISS に追加された件数:", index.ntotal)



FAISS に追加された件数: 8847


In [134]:
# 類似検索テスト
query = model.encode(test.iloc[0]["Sentences"], convert_to_numpy=True).astype('float32')

k = 45
distances, indices = index.search(query.reshape(1, -1), k)

print(indices)
print(distances)


[[3476 7344 8336 8522 3740 3330 3608 8263 7424 6051 8149 1204  787 6684
  1890 8404 8176 7827 7625  294 2623 4198 1668 7548 8406 6351 7194 1257
  7559 8083 5671 3021 2618  207 1062 6374 6931  372 6750 4887 4225 8606
  4297 4574  666]]
[[0.8448929  0.91808504 0.9404224  0.954873   0.9601603  0.9618082
  0.9806144  1.0138112  1.0184844  1.0193832  1.0212893  1.0234567
  1.0257134  1.0282106  1.0334785  1.0398726  1.0413783  1.0471346
  1.0475119  1.0500329  1.052748   1.0556294  1.059175   1.0613909
  1.0615077  1.0630999  1.0634594  1.065839   1.0662494  1.0664468
  1.0667781  1.0716722  1.0758204  1.0766556  1.0810692  1.0826294
  1.0846196  1.0865464  1.0866022  1.0915204  1.0937612  1.0963136
  1.0974312  1.1004199  1.1015258 ]]


In [135]:
import re, pandas as pd

def base_tid(x):
    if pd.isna(x): return None
    m = re.search(r"(T\d{4})", str(x))
    return m.group(1) if m else None

K = 1
strict_hit = 0
parent_hit = 0

for _, row in test.iterrows():
    q = model.encode(row["Sentences"], convert_to_numpy=True).astype("float32")
    _, I = index.search(q.reshape(1, -1), K)

    pred = docs[I[0][0]].metadata["TTP_ID"]
    gold = row["act_Technique ID"]

    if str(gold) == str(pred):
        strict_hit += 1
    if base_tid(gold) == base_tid(pred):
        parent_hit += 1

print("Recall@1 strict :", strict_hit/len(test))
print("Recall@1 parent :", parent_hit/len(test))



Recall@1 strict : 0.5184743742550656
Recall@1 parent : 0.5184743742550656


In [136]:
test["act_Technique ID"].astype(str).str.contains(r"\.", regex=True).mean()


np.float64(0.0)

In [137]:
import re
import pandas as pd

def base_tid(x):
    if pd.isna(x):
        return None
    m = re.search(r"(T\d{4})", str(x))
    return m.group(1) if m else None

Ks = [1,45]
results = {}

for K in Ks:
    recall_hit = 0

    for _, row in test.iterrows():
        query_vec = model.encode(row["Sentences"], convert_to_numpy=True).astype("float32")
        distances, indices = index.search(query_vec.reshape(1, -1), K)

        retrieved_ttps = [docs[i].metadata["TTP_ID"] for i in indices[0]]

        gold_base = base_tid(row["act_Technique ID"])
        retrieved_base_set = {base_tid(t) for t in retrieved_ttps}

        if gold_base in retrieved_base_set:
            recall_hit += 1

    recall_k = recall_hit / len(test)
    results[K] = recall_k
    print(f"Recall@{K:3d} (parent-match) = {recall_k:.4f}")


Recall@  1 (parent-match) = 0.5185
Recall@ 45 (parent-match) = 0.8570


In [138]:
df

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
0,1,train,T1543,Create or Modify System Process: Windows Service,Persistence:Privilege Escalation,Carbanak malware installs itself as a service ...,NaN
1,2,train,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,Carbanak may use netsh to add local firewall r...,NaN
2,3,train,T1036,Masquerading: Masquerade Task or Service,Defense Evasion,Carbanak has copied legitimate service names t...,NaN
3,4,train,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,"Carbanak has named malware ""svchost.exe,"" whic...",NaN
4,5,train,T1588,Obtain Capabilities: Tool,Resource Development,Carbanak has obtained and used open-source too...,NaN
...,...,...,...,...,...,...,...
8382,8383,train,T1573,Encrypted Channel: Symmetric Cryptography,Command and Control,NETEAGLE will decrypt resources it downloads w...,NaN
8383,8384,train,T1041,Exfiltration Over C2 Channel,Exfiltration,NETEAGLE is capable of reading files over the ...,NaN
8384,8385,train,T1083,File and Directory Discovery,Discovery,NETEAGLE allows adversaries to enumerate and m...,NaN
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Exfiltration


In [139]:
df2

,Key,Value
0,T1094,Some Software variants use raw TCP for C2.
1,T1047,Software leverages WMI to enumerate anti-virus...
2,T1003.001,Software contains an implementation of Softwar...
3,T1614.001,Attacker has used implants to collect the syst...
4,T1587.001,For Software modular backdoor.
...,...,...
19742,T1045,Most decompression techniques decompress the e...
19743,T1059.001,Software uses PowerShell to download and execu...
19744,T1059.003,Software can use `cmd.exe` to execute a create...
19745,T1112,Software has functionality to remove Registry ...


# 生成テスト用（TacticsのBERT)

In [140]:
from collections import Counter
import numpy as np
import re
import pandas as pd

K = 45

number = 3
a = number - 1
row = test.iloc[a]

query_text = row["Sentences"]
true_tid   = row["act_Technique ID"]
true_tac   = row.get("act_TacticsName")

def norm_tid(x):
    if pd.isna(x):
        return None
    m = re.search(r"(T\d{4})", str(x))
    return m.group(1) if m else str(x)

true_tid_n = norm_tid(true_tid)

query_vec = model.encode(query_text, convert_to_numpy=True).astype("float32")
distances, indices = index.search(query_vec.reshape(1, -1), K)

top_k_indices = indices[0]
top_k_dists   = distances[0]

retrieved_tids   = [docs[i].metadata.get("TTP_ID") for i in top_k_indices]
retrieved_tids_n = [norm_tid(t) for t in retrieved_tids]

print(f"===== {number}件目のチェック =====")
print(f"Technique ID (真値) : {true_tid}  -> 正規化: {true_tid_n}")
if true_tac is not None:
    print(f"Tactics (真値)      : {true_tac}")
print()
print(f"入力CTI文:\n{query_text}\n")

# 親IDで最初に出てきたものだけ残す（順位は維持）
seen = set()
unique_by_parent = []
for rank, (idx_i, dist) in enumerate(zip(top_k_indices, top_k_dists), start=1):
    meta = docs[idx_i].metadata
    tid = meta.get("TTP_ID")
    p = norm_tid(tid)
    if p in seen:
        continue
    seen.add(p)
    unique_by_parent.append((rank, idx_i, dist, tid))

print(f"上位 {K} 件から抽出 → 親IDで重複除去後 {len(unique_by_parent)} 件:")
for new_rank, (orig_rank, idx_i, dist, tid) in enumerate(unique_by_parent, start=1):
    meta = docs[idx_i].metadata
    print(f"{new_rank:2d}. (orig#{orig_rank:2d}) ID={tid}  "
          f"Name={meta.get('TechniqueName','')}  "
          f"Tactics={meta.get('Tactics','')}  "
          f"dist={dist:.4f}")

print()
print(f"親ID一致（サブOK） -> {true_tid_n in set(retrieved_tids_n)}")
print(f"完全一致（参考）   -> {str(true_tid) in set(map(str, retrieved_tids))}")


===== 3件目のチェック =====
Technique ID (真値) : T1566  -> 正規化: T1566
Tactics (真値)      : Initial Access

入力CTI文:
Frankenstein has used spearphishing emails to send trojanized Microsoft Word documents.[1]

上位 45 件から抽出 → 親IDで重複除去後 19 件:
 1. (orig# 1) ID=T1593  Name=  Tactics=  dist=0.7024
 2. (orig# 2) ID=T1221  Name=  Tactics=  dist=0.7498
 3. (orig# 3) ID=T1192  Name=  Tactics=  dist=0.8042
 4. (orig# 4) ID=T1204  Name=  Tactics=  dist=0.8182
 5. (orig# 5) ID=T1105  Name=  Tactics=  dist=0.8185
 6. (orig# 6) ID=T1587  Name=  Tactics=  dist=0.8255
 7. (orig# 7) ID=T1193  Name=  Tactics=  dist=0.8817
 8. (orig# 8) ID=T1566  Name=  Tactics=  dist=0.8835
 9. (orig#11) ID=T1203  Name=  Tactics=  dist=0.9052
10. (orig#12) ID=T1534  Name=  Tactics=  dist=0.9127
11. (orig#14) ID=T1083  Name=  Tactics=  dist=0.9226
12. (orig#15) ID=T1583  Name=  Tactics=  dist=0.9276
13. (orig#16) ID=T1114  Name=  Tactics=  dist=0.9406
14. (orig#18) ID=T1078  Name=  Tactics=  dist=0.9421
15. (orig#21) ID=T1039  Name= 

In [141]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    model_kwargs={"device": "cuda:1"} #GPU指定
)


In [142]:
from langchain_community.vectorstores import FAISS as LCFAISS
db = LCFAISS.from_documents(docs, embeddings)
db.save_local("TTPHunter_50_TTPs_andTactics_2")

In [143]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS as LCFAISS

# 1. 保存時と同じモデル名で embeddings（モデル）を復元
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    model_kwargs={"device": "cuda:1"} #GPU指定
    # SecureBERT 使って保存したなら、ここも同じ名前にする
)

# 2. ローカルからベクターストアをロード
db = LCFAISS.load_local(
    "TTPHunter_50_TTPs_andTactics_2",
    embeddings,
    allow_dangerous_deserialization=True
)

print("loaded FAISS index:", db)

docs = db.similarity_search("test", k=1)
doc = docs[0]
print(doc.metadata)  # ここに "Tactics" が見えてたらOK


loaded FAISS index: <langchain_community.vectorstores.faiss.FAISS object at 0x7efddc307910>
{'TTP_ID': 'T1086'}


In [144]:
test

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
14,15,test,T1005,Data from Local System,Collection,"Frankenstein has enumerated hosts via Empire, ...",Discovery
20,21,test,T1588,Obtain Capabilities: Tool,Resource Development,Frankenstein has obtained and used Empire to d...,Execution
22,23,test,T1566,Phishing: Spearphishing Attachment,Initial Access,Frankenstein has used spearphishing emails to ...,Initial Access
23,24,test,T1057,Process Discovery,Discovery,"Frankenstein has enumerated hosts, looking to ...",Discovery
29,30,test,T1204,User Execution: Malicious File,Execution,Frankenstein has used trojanized Microsoft Wor...,Execution
...,...,...,...,...,...,...,...
8368,8369,test,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,TYPEFRAME can open the Windows Firewall on the...,Defense Evasion
8369,8370,test,T1070,Indicator Removal on Host: File Deletion,Defense Evasion,TYPEFRAME can delete files off the system.[1],Defense Evasion
8381,8382,test,T1059,Command and Scripting Interpreter: Windows Com...,Execution,NETEAGLE allows adversaries to execute shell c...,Execution
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Exfiltration


### Few-Shot Learning

In [145]:
import sys
print(sys.executable)


/workspace/miluser/Mitsuharu_2002/mitsuharu-12-02/bin/python


In [146]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/workspace/miluser/Mitsuharu_2002


# llama 読み込み

In [147]:
# !pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="unsloth/Llama-3.3-70B-Instruct-GGUF",
	filename="Llama-3.3-70B-Instruct-Q5_K_M.gguf",
	cache_dir = "/workspace/miluser/Mitsuharu_2002/llama_cache",
    n_ctx=4000,
    n_gpu_layers=-1,          # できるだけ GPU に載せる
    main_gpu=0,               # 小さいテンソルは GPU0 で処理
    tensor_split=[0.5, 0.5],  # 大きいテンソルを GPU0/1 に 50:50 で分割
    verbose=False,             # ロードログを出す
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)


llama_context: n_ctx_per_seq (4000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


{'id': 'chatcmpl-3cc4a879-9dc1-451f-a9ca-f3e85670fb6b',
 'object': 'chat.completion',
 'created': 1766547364,
 'model': '/workspace/miluser/Mitsuharu_2002/llama_cache/models--unsloth--Llama-3.3-70B-Instruct-GGUF/snapshots/8f14c5c5d06fca109ff16f94496147a8479711de/./Llama-3.3-70B-Instruct-Q5_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'The capital of France is Paris.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 42, 'completion_tokens': 7, 'total_tokens': 49}}

In [148]:
df

,ID,test_or_train,act_Technique ID,act_Technique Name,act_TacticsName,Sentences,Predicted_TacticsName
0,1,train,T1543,Create or Modify System Process: Windows Service,Persistence:Privilege Escalation,Carbanak malware installs itself as a service ...,NaN
1,2,train,T1562,Impair Defenses: Disable or Modify System Fire...,Defense Evasion,Carbanak may use netsh to add local firewall r...,NaN
2,3,train,T1036,Masquerading: Masquerade Task or Service,Defense Evasion,Carbanak has copied legitimate service names t...,NaN
3,4,train,T1036,Masquerading: Match Legitimate Name or Location,Defense Evasion,"Carbanak has named malware ""svchost.exe,"" whic...",NaN
4,5,train,T1588,Obtain Capabilities: Tool,Resource Development,Carbanak has obtained and used open-source too...,NaN
...,...,...,...,...,...,...,...
8382,8383,train,T1573,Encrypted Channel: Symmetric Cryptography,Command and Control,NETEAGLE will decrypt resources it downloads w...,NaN
8383,8384,train,T1041,Exfiltration Over C2 Channel,Exfiltration,NETEAGLE is capable of reading files over the ...,NaN
8384,8385,train,T1083,File and Directory Discovery,Discovery,NETEAGLE allows adversaries to enumerate and m...,NaN
8385,8386,test,T1057,Process Discovery,Discovery,NETEAGLE can send process listings over the C2...,Exfiltration


In [149]:
import re
from llama_cpp import Llama   # もう import 済みなら不要


# # ★ llm はループの外で1回だけ作る
# llm = Llama.from_pretrained(
#     repo_id="unsloth/Llama-3.3-70B-Instruct-GGUF",
#     filename="Llama-3.3-70B-Instruct-Q8_0/Llama-3.3-70B-Instruct-Q8_0-00001-of-00002.gguf",
#     # オプション（必要に応じて）
#     n_ctx=4000,
#     n_gpu_layers=-1,  # GPU 使うなら。全部GPUに乗せるなら -1
#     # main_gpu=1,     # GPU1番を使いたいならこういう指定もアリ（バージョンによる）
#     verbose=False, 
# )

predicted_labels = []

for index, row in test.iterrows():
    text_to_predict = row["Sentences"]
    id = row["ID"]

    # 1. MiniLM + FAISS で 65件取得（few-shot数）
    search_results = db.similarity_search_with_score(text_to_predict, k=55)

    # 2. score < 2 のものだけ例として採用
    xs = [
        "\nText:" + doc.page_content.lower().strip() +
        "\nSimilarity:" + str(score) + "\n" +
        "Classification:" + doc.metadata['TTP_ID'] + "\n"
        for doc, score in search_results
        if score < 2
    ]

    examples_block = ''.join(xs)

    prompt = (
        "You are a cybersecurity expert.\n"
        "Your task is to classify the last sentence of 'InputText' based on the provided examples.\n"
        "Each example shows a sentence, a similarity score (smaller value means more similar), "
        "and its corresponding classification label.\n"
        "Rules:\n"
        "- You MUST answer with exactly ONE classification label.\n"
        "- The label MUST start with 'T' followed by 4 digits (e.g., T1082).\n"
        "- Do NOT output '?', explanations, or anything else.\n"
        "- If you are unsure, choose the most plausible label from the patterns in the examples.\n\n"
        "Examples:\n"
        f"{examples_block}\n"
        f"InputText:{text_to_predict.lower().strip()}\n"
        "Classification:"
    )

    # ★ llama.cpp / unsloth のチャットAPIだけに統一する
    out = llm.create_chat_completion(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        temperature=0.0,
        max_tokens=16,  # ラベルだけ返ってくればいいので小さめでOK
    )

    # ★ chat形式なので "message"]["content"] に入ってる
    raw = out["choices"][0]["message"]["content"]

    # 3. 出力から Txxxx を正規表現で抜き出す
    m = re.search(r"T\d{4}", raw)
    if m:
        predicted_label = m.group(0)
    else:
        # 方針次第でここをどうするか決める
        predicted_label = ""
        # 例えば最近傍にフォールバックするならこう（使うならコメント外す）
        # best_doc, best_score = min(search_results, key=lambda x: x[1])
        # predicted_label = best_doc.metadata['TTP_ID']

    predicted_labels.append(predicted_label)
    print("ID:", id, ", actual:", row["act_Technique ID"], ", predicted:", predicted_label)


ID: 15 , actual: T1005 , predicted: T1082
ID: 21 , actual: T1588 , predicted: T1105
ID: 23 , actual: T1566 , predicted: T1221
ID: 24 , actual: T1057 , predicted: T1057
ID: 30 , actual: T1204 , predicted: T1221
ID: 32 , actual: T1047 , predicted: T1047
ID: 33 , actual: T1087 , predicted: T1069
ID: 34 , actual: T1087 , predicted: T1069
ID: 39 , actual: T1547 , predicted: T1112
ID: 40 , actual: T1059 , predicted: T1059
ID: 43 , actual: T1005 , predicted: T1005
ID: 45 , actual: T1041 , predicted: T1041
ID: 49 , actual: T1036 , predicted: T1036
ID: 50 , actual: T1036 , predicted: T1036
ID: 51 , actual: T1027 , predicted: T1027
ID: 52 , actual: T1588 , predicted: T1003
ID: 66 , actual: T1078 , predicted: T1078
ID: 71 , actual: T1105 , predicted: T1105
ID: 73 , actual: T1553 , predicted: T1116
ID: 86 , actual: T1553 , predicted: T1553
ID: 91 , actual: T1087 , predicted: T1087
ID: 99 , actual: T1555 , predicted: T1003
ID: 120 , actual: T1021 , predicted: T1133
ID: 125 , actual: T1016 , predict

In [151]:
print("len(test)            =", len(test))
print("len(predicted_labels) =", len(predicted_labels))
# 予測結果をDataFrameに追加
test["Predicted"] = predicted_labels

# CSVとして保存
test.to_csv("DSAサブテクニック削除版.csv", index=False, encoding="utf-8")


len(test)            = 1678
len(predicted_labels) = 1678


In [152]:
print(test.columns)



Index(['ID', 'test_or_train', 'act_Technique ID', 'act_Technique Name',
       'act_TacticsName', 'Sentences', 'Predicted_TacticsName', 'Predicted'],
      dtype='object')


### Evaluate

In [153]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 正解ラベル（ATT&CK Technique ID）
y_true = test["act_Technique ID"].values

# 予測ラベル（Llama の出力）
# predicted_labels は Llama 推論ループで append していたリスト
y_pred = predicted_labels

# ---- 評価 ----
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"Accuracy:  {accuracy}")
print(f"Precision: {precision}")
print(f"Recall:    {recall}")
print(f"F1 Score:  {f1}")


Accuracy:  0.5953516090584029
Precision: 0.8129904697792738
Recall:    0.5953516090584029
F1 Score:  0.6137735229492881


In [ ]:
# import pandas as pd

# # ① CSV を読み込み
# df_base = pd.read_csv("/workspace/miluser/Mitsuharu_2002/csv_data/TTPHunter_dataset.csv")              # Domain, Technique ID, Technique Name, Sentences
# df_pred = pd.read_csv("/workspace/miluser/Mitsuharu_2002/predicted_Tactics_70B_1678.csv")     # Technique ID, Tactics Name, Sentences, Predicted

# # （もし念のためヘッダに余計な空白がありそうなら）
# df_base.columns = df_base.columns.str.strip()
# df_pred.columns = df_pred.columns.str.strip()

# # ② Predicted だけ使う（＋キーになる Sentences）
# df_pred_small = df_pred[["Sentences", "Predicted"]]

# # ③ Sentences をキーに結合（LEFT JOIN）
# df_merged = df_base.merge(df_pred_small, on="Sentences", how="left")

# # ④ 保存
# df_merged.to_csv("TTPHunter_with_Predicted.csv", index=False)


In [ ]:
["\nClassification:"+str(score)+"\n" for doc, score in search_results]

以下追加分（混合行列）

In [ ]:
import sys
print(sys.executable)

!pip install --upgrade pip


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred, labels=sorted(list(set(y_true))))
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=True, yticklabels=True)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

import pandas as pd

print(pd.Series(y_pred).value_counts())



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# ラベルを揃えておく
labels = sorted(list(set(y_true)))

cm = confusion_matrix(y_true, y_pred, labels=labels)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,      # ★ 数字を表示
    fmt="d",         # ★ 整数表示（floatなら ".2f" とか）
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print(pd.Series(y_pred).value_counts())


In [ ]:
for t,p,txt in zip(y_true, y_pred, test["text"]):
    if t != p:
        print("True:", t)
        print("Pred:", p)
        print("Text:", txt)
        print("-" * 50)



### Few Shot learning 2

In [ ]:
# Aggregate predictions using majority voting
from collections import Counter

def majority_voting(predictions):
    count = Counter(predictions)
    return count.most_common(1)[0][0]

In [ ]:
import random

def create_prompt(template, search_results, text_to_predict):
    xs = ["\nText:"+ doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID'] for doc, _ in search_results]
    xs = ''.join(xs)+"\nText:"+ text_to_predict.lower() +"\nClassification:"
    return template + xs

def split_list(lst):
    random.shuffle(lst)  # Shuffle the list to ensure randomness
    n = len(lst)
    third = n // 3
    remainder = n % 3

    # Determine the splitting points
    split1 = third
    split2 = third * 2

    if remainder == 1:
        split1 += 1
    elif remainder == 2:
        split1 += 1
        split2 += 1

    # Split the list
    first_third = lst[:split1]
    second_third = lst[split1:split2]
    third_third = lst[split2:]

    return first_third, second_third, third_third



predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    search_results = db.similarity_search_with_score(text_to_predict, k=150)

    template1 = "Your task is to classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n"
    template2 = "Please classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its classification label. Use the patterns from these examples to classify the given text correctly. Return only the classification label starting with 'T'.\n"
    template3 = "Your task is to determine the classification of the last sentence of 'Text' using the provided examples. Each example pairs a sentence with its classification label. Analyze these examples to classify the given text. Output only the classification label starting with 'T'.\n"

    ## Split the list
    first_part, second_part, third_part = split_list(search_results)

    
    prompt1 = create_prompt(template1, first_part, text_to_predict)
    prompt2 = create_prompt(template2, second_part, text_to_predict)
    prompt3 = create_prompt(template3, third_part, text_to_predict)
    
    # Collect predictions
    prompts = [prompt1, prompt2, prompt3]
    predictions = []
    
    for prompt in prompts:
        
        prediction = llm.invoke(prompt).content
        predictions.append(prediction)

    print(predictions)
    
    
    final_prediction = majority_voting(predictions)
    #print("Final Prediction:", final_prediction)
    predicted_labels.append(final_prediction)
    print("actual:", row["Technique ID"], ", predicted:", final_prediction)

### Evaluate

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

temperature:0, k=45

Accuracy: 0.9523809523809523
Precision: 0.9761904761904762
Recall: 0.9523809523809523
F1 Score: 0.9603174603174602

temperature:1, k=45

Accuracy: 0.9285714285714286
Precision: 0.9642857142857143
Recall: 0.9285714285714286
F1 Score: 0.9365079365079364

### V2 - SecureBERT Embeddings

In [ ]:
### embedding
model2 = "ehsanaghaei/SecureBERT"
embeddings2 = HuggingFaceEmbeddings(model_name = model2)

docs2 = []
for index, row in train.iterrows():
    doc = Document(page_content=row["Sentences"], metadata={"TTP_ID": row["Technique ID"]})
    docs2.append(doc)

print(len(docs2))

In [ ]:
db2 = FAISS.from_documents(docs2, embeddings2)


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings3 = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

db3 = FAISS.from_documents(docs2, embeddings3)


In [ ]:
db2

In [ ]:
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="o1-mini", temperature=1)
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    ##### Prompt 1
    search_results = db3.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    [scores_list.append(score) for doc, score in search_results]
    ### search
    '''
    search_results = db.similarity_search_with_score(text_to_predict, k=55)
    xs = ["\n"+ "Example "+ str(i) +":\nText:"+ doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID']+"\n" for i, (doc, _) in enumerate(search_results)]
    ##xs = ["\nText:"+" ".join(str(doc.page_content).lower().split())+"\nClassification:"+doc.metadata['TTP_ID'] for doc, _ in search_results]
    

    ### mmr search
    #search_results = db.max_marginal_relevance_search(text_to_predict,k=45, fetch_k=250)
    #xs = ["\nText:"+doc.page_content.lower()+"\nClassification:"+doc.metadata['TTP_ID'] for doc in search_results]

    
    xs += "\n\nNote: The examples show a pattern where certain attack behaviors correspond to specific classification labels. Use this pattern recognition to classify the new text accurately.\nNow, classify the following text:\n"
    xs = ''.join(xs)+"\nText:"+ text_to_predict.lower() +"\nClassification:"
    #xs = "Your task is to classify the last sentence of 'Text' based on the provided examples. Each example shows a sentence and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    xs = "Your task is to classify the last sentence of 'Text' based on the diverse examples provided below. Learn the patterns from these examples to make accurate classifications. Output only the classification label beginning with 'T'.\n\nExamples:\n" + xs
    '''
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

### No LLM Call

In [ ]:
from langchain_openai import ChatOpenAI

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    ##### Prompt 1
    search_results = db2.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    [scores_list.append(score) for doc, score in search_results]
    
    #print(xs)
    #prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    #result = llm.invoke(prompt)
    #predicted_label = result.content
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    #predicted_labels.append(predicted_label)
    #print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

In [ ]:
### Evaluate

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
# Secure Bert Embeddings - 
Accuracy: 0.9523809523809523
Precision: 0.9880952380952381
Recall: 0.9523809523809523
F1 Score: 0.9619047619047618

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
# Find incorrectly classified labels
incorrect_classifications = []
for index, (true_label, pred_label) in enumerate(zip(y_true, y_pred)):
    if true_label != pred_label:
        incorrect_classifications.append({
            'Index': index,
            'True Label': true_label,
            'Predicted Label': pred_label
        })

# Output the results
if incorrect_classifications:
    print("Incorrectly Classified Labels:")
    for item in incorrect_classifications:
        print(f"Index {item['Index']}: True Label = {item['True Label']}, Predicted Label = {item['Predicted Label']}")
else:
    print("All labels are correctly classified.")

In [ ]:
print(xs)

In [ ]:
train["Technique ID"].value_counts()["T1036"]#.head()

### SImilarity Investigation

In [ ]:
score_list = []
very_similar_string = "kerrdown can use a vbs base64 decoder function published by motobit"
totaly_wrong_string = "we have a nice weather today"
search_results2 = db2.similarity_search_with_score(very_similar_string, k=45)
[score_list.append(score) for doc, score in search_results2]
["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results2 if score < 2]
#score_list

### V3

In [ ]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
#llm = ChatOpenAI(model="chatgpt-4o-latest", temperature=0)

#llm.invoke(prompt)

def predict_label(text_to_predict):
    ##### Prompt 1
    search_results = db2.similarity_search_with_score(text_to_predict, k=45)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are a MITRE ATT&CK TTP classification expert.
    Your task is to classify the 'InputText' based on the provided examples below.
    Each example shows a sentence, inverse similarity to InputText (the lower the better),
    and its corresponding classification label starting with 'T'.
    Use these examples to determine the correct classification for the given text (InputText).
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class for 'InputText'.\n""" + xs
    #xs = "You are an expert in text classification. Below are examples of texts with their classifications based on their similarity to the 'InputText'. For the 'InputText' provided, choose the most appropriate classification based on the examples given. Only return the classification label (e.g., TXXXX). Use the closest similarity as the primary criterion for classification.\n\n" + xs
    #####
    
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content
    return predicted_label


scores_list = []
predicted_labels = []
# Iterating over the DataFrame row by row
for index, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    predicted_label = predict_label(text_to_predict)
    
    #predicted_label = most_common_ttp_id
    #print(predicted_label)
    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    #break

In [ ]:
print(xs)

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
predict_label("We have a beautiful wether today")

### Text to Predict

In [ ]:
text_to_predict = """
sofacy uses dealerschoice to target european government agency
robert falcone	by robert falcone
march 15, 2018 at 1:00 pm
category: unit 42 tags: dealerschoice, european government agency, sofacy summary back in october 2016, unit 42 published an initial analysis on a flash exploitation framework used by the sofacy threat group called dealerschoice. the attack consisted of microsoft word delivery documents that contained adobe flash objects capable of loading additional malicious flash objects embedded in the file or directly provided by a command and control server. sofacy continued to use dealerschoice throughout the fall of 2016, which we also documented in our december 2016 publication discussing sofacy s larger campaign. on march 12 and march 14, we observed the sofacy group carrying out an attack on a european government agency involving an updated variant of dealerschoice. the updated dealerschoice documents used a similar process to obtain a malicious flash object from a c2 server, but the inner mechanics of the flash object contained significant differences in comparison to the original samples we analyzed. one of the differences was a particularly clever evasion technique: to our knowledge this has never been observed in use. with the previous iterations of dealerschoice samples, the flash object would immediately load and begin malicious tasks. in the march attacks, the flash object is only loaded if the user scrolls through the entire content of the delivery document and views the specific page the flash object is embedded on. also, dealerschoice requires multiple interactions with an active c2 server to successfully exploit an end system. the overall process to result in a successful exploitation is: user must open the microsoft word email attachment
user must scroll to page three of the document, which will run the dealerschoice flash object
the flash object must contact an active c2 server to download an additional flash object containing exploit code
the initial flash object must contact the same c2 server to download a secondary payload
victim host must have a vulnerable version of flash installed the attack the attack involving this updated variant of dealerschoice was targeting a european government organization. the attack relied on a spear-phishing email with a subject of defence & security 2018 conference agenda that had an attachment with a filename of defence & security 2018 conference . the attached document contains a conference agenda that the sofacy group appears to have copied directly from the website for the underwater defence & security 2018 conference here. opening the attached defence & security 2018 conference file does not immediately run malicious code to exploit the system. instead, the user must scroll to the third page of the document, which will load a flash object that contains actionscript that will attempt to exploit the user s system to install a malicious payload. the flash object embedded within this delivery document is a variant of an exploit tool that we call dealerschoice. this suggests that the sofacy group is confident that the targeted individuals would be interested enough in the content to peruse through it. we analyzed the document to determine the reason that the malicious flash object only ran when the user scrolled to the third page. according to the file, the dealerschoice loader swf exists after the covert-shores- image file within the delivery document. this image file exists on the third page of the document, so the user would have to scroll down in the document to this third page to get the swf file to run. the user may not notice the flash object on the page, as word displays it as a tiny black box in the document, as seen in figure 1. this is an interesting anti-sandbox technique, as it requires human interaction prior to the document exhibiting any malicious activity. updated dealerschoice this dealerschoice flash object shares a similar process to previous variants; however, it appears that the sofacy actors have made slight changes to its internal code. also, it appears that the actors used actionscript from an open source video player called f4player , which is freely available on github with the following description: f4player is an open source flash (as3) video player and library project. it is so small that it is only 10kb (with skin file) and totally free under gpl license. the sofacy developer modified the f4player s actionscript to include additional code to load an embedded flash object. the additions include code to decrypt an embedded flash object and an event handler that calls a newly added function ( skinevent2 ) that plays the decrypted object, as seen in the code snippet below: the above code allows dealerschoice to load a second swf object, specifically loading it with an argument that includes a c2 url of hxxp://ndpmedia24[.]com/0pq6m4f.m3u8 . the embedded swf extracts the domain from the c2 url passed to it and uses it to craft a url to get the server s file in order to obtain permissions to load additional flash objects from the c2 domain. the actionscript relies on event listeners to call specific functions when the event event.complete is triggered after successful http requests are issued to the c2 server. the event handlers call functions with the following names, which includes an incrementing number that represents the order in which the functions are called: with these event handlers created, the actionscript starts by gathering system data from the flash.system.capabilities.serverstring property (just like in the original samples) and issues an http get with the system data as a parameter to the c2 url that was passed as an argument to the embedded swf when it was initially loaded. when this http request completes, the event listener will call the onload1 function. the onload1 function parses the response data from the request to the c2 url using regular expressions. according to the following code snippet, it appears the regular expression is looking for a hexadecimal string after / and before /sec , as well as any string between /hls/ and /tracks : the regular expressions suggest that the c2 server responds with content that is meant to resemble http live steaming (hls) traffic, which is a protocol that uses http to deliver audio and video files for streaming. the use of hls coincides with the use of actionscript code from the f4player to make the traffic seem legitimate. the variables storing the results of the regular expression matches are used within the actionscript for further interaction with the c2 server. the following is a list of these variables and their purpose: the onload1 function then sends an http get request to the c2 domain using the value stored in the r3 variable as a url. when this http request completes, the event listener will call the onload2 function. the onload2 function decrypts the response received from the http request issued in onload1 function. it does so by calling a sub-function to decrypt the content, using the value stored in the r1 variable as a key. the sub-function to decrypt the content skips the first 4 bytes, suggesting that the first four bytes of the downloaded content is in cleartext (most likely the fws or cws header to look legitimate). after decrypting the content, the onload2 function will issue another http get request with the system data as a parameter, but this time to the c2 using a url from the r4 variable. when this request completes, the event listener will call the onload3 function. the onload3 function will take the response to the http request in onload2 and treat it as the payload. the actionscript will read each byte of the c2 response and get the hexadecimal value of each byte and create a text array of 4-byte hexadecimal values with 0x prepended and , appended to each using the following code: this hexadecimal string will most likely be a string of shellcode that will contain and decrypt the ultimate portable executable (pe) payload. the string of comma separated hexadecimal values is passed as a parameter when loading the swf file downloaded in onload2 . this function creates an event listener for when the swf file is successfully loaded, which will call the onload5 function. the onload5 function is responsible for adding the newly loaded swf object as a child object to the current running object using the following code: this loads the swf file, effectively running the malicious code on the system. during our analysis, we were unable to coerce the c2 into providing a malicious swf or payload. as mentioned in our previous blogs on dealerschoice, the payload of choice for previous variants was sofacycarberp (seduploader), but we have no evidence to suggest this tool was used in this attack. we are actively researching and will update this blog in the event we discover the malicious flash object and payload delivered in this attack. linkage to prior campaign the delivery document used in this attack was last modified by a user named nick daemoji , which provides a linkage to previous sofacy related delivery documents. the previous documents that used this user name were macro-laden delivery documents that installed sofacycarberp/seduploader payloads, as discussed in talos blog. this overlap also points to a similar social engineering theme between these two campaigns, as both used content from upcoming military and defense conferences as a lure. conclusion the sofacy threat group continues to use their dealerschoice framework to exploit flash vulnerabilities in their attack campaigns. in the most recent variant, sofacy modified the internals of the malicious scripts, but continues to follow the same process used by previous variants by obtaining a malicious flash object and payload directly from the c2 server. unlike previous samples, this dealerschoice used a docx delivery document that required the user to scroll through the document to trigger the malicious flash object. the required user interaction turned out to be an interesting anti-sandbox technique that we had not seen this group perform in the past.
"""

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text = "Hello world. How are you? I'm doing great!"
sentences = sent_tokenize(text)

print(sentences)

In [ ]:
###
text_sentences = sent_tokenize(text_to_predict)

text_ttps = []
for sentence in text_sentences:
    text_ttp = predict_label(sentence)
    print(sentence, " ->", text_ttp, "\n---")
    text_ttps.append(text_ttp)

In [ ]:
set(text_ttps)

In [ ]:
### 
predict_label("Process Hashes and Process Privilege Checks Upon being loaded, the malware starts a subroutine that hashes each running process on the system, and compares each hash with 3 hardcoded hashes: 0x6403527E → avp.exe associated with Kaspersky AV 0x23214B44 → ns.exe associated with Norton Security 0x651B3005 → ccSvcHst.exe associated with Symantec A global flag PROC_FLAG is used to track which of the 3 processes are running on the system. PROC_FLAG")

In [ ]:
predict_label("PROC_FLAG")

### V4 - DB Index Optimization

In [ ]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans

# Load SecureBERT
tokenizer = AutoTokenizer.from_pretrained(model2)
model = AutoModel.from_pretrained(model2)

# Move model to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate SecureBERT embeddings for a single text
def get_embedding(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**tokens)
    # Pool the last hidden state to get sentence embedding
    embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move embedding to CPU
    return embedding

# Extract texts and labels
texts = list(train["Sentences"].values)  # List of text items
labels = list(train["Technique ID"].values)  # List of labels

# Get SecureBERT embeddings for each document, item by item
embeddings = []

for text in texts:
    embedding = get_embedding(text)
    embeddings.append(embedding)

# Convert list of embeddings to a numpy array
embeddings = np.vstack(embeddings)  # Stack embeddings vertically to create a single array

# Normalize embeddings (L2 normalization)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)





In [ ]:
# Normalize embeddings (L2 normalization)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Supervised clustering using labels

# Get unique labels
unique_labels = set(labels)
n_classes = len(set(labels))
label_embeddings = {label: [] for label in unique_labels}

# Group embeddings by label
for embedding, label in zip(embeddings, labels):
    label_embeddings[label].append(embedding)

# Create label-guided centroids using KMeans for each class
centroids = []
for label, emb_list in label_embeddings.items():
    emb_array = np.array(emb_list)
    if len(emb_list) > 1:
        # Use KMeans to create centroids for each class, useful if there are multiple samples per class
        kmeans = KMeans(n_clusters=1, random_state=42)
        kmeans.fit(emb_array)
        centroids.append(kmeans.cluster_centers_[0])
    else:
        # If only one sample, use it directly as the centroid
        centroids.append(emb_array[0])

# Convert centroids to a numpy array
centroids = np.array(centroids)

# Initialize FAISS Index
d = embeddings.shape[1]  # dimension of the embeddings
nlist = n_classes  # number of clusters, one per class

#quantizer = faiss.IndexFlatL2(d)  # the quantizer used for clustering
#index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

index = faiss.IndexFlatL2(d)

# Train the FAISS index with the supervised centroids
#index.train(centroids)
#index.train(embeddings)
# Add embeddings to the index
index.add(embeddings)



In [ ]:
centroids

In [ ]:
# Example query


query_texts = ["Search the document with relevant content."]
query_embeddings = get_embedding(query_texts)
query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)

# Perform search
k = 45  # number of nearest neighbors
distances, indices = index.search(query_embeddings, k)

print(distances)

'''
# Print the search results
for i, query in enumerate(query_texts):
    print(f"Query: {query}")
    for j in range(k):
        if indices[i][j] != -1:  # valid index check
            print(f"Neighbor {j+1}: Document: {documents[indices[i][j]]['text']}, Distance: {distances[i][j]}")
'''

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)###, model_kwargs={"top_p": 0.0})
predicted_labels = []
scores_list = []
for i, row in test.iterrows():
    #print(f"Index: {index}")
    text_to_predict = row["Sentences"]

    query_texts = text_to_predict#["Search the document with relevant content."]
    query_embeddings = get_embedding(query_texts)
    query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
    
    # Perform search
    k = 60  # number of nearest neighbors
    distances, indices = index.search(query_embeddings, k)

    retrieved_docs = [texts[i] for i in indices[0] if i != -1]
    retrieved_labels = [labels[i] for i in indices[0] if i != -1]


    
    #[print(doc, score, label) for doc, score, label in zip(retrieved_docs, distances, labels) if score < 2]
    
    xs = ["\nText:"+ doc.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+label+"\n" for doc, score, label in zip(retrieved_docs, distances.flatten(), retrieved_labels) if score < 2]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = "Your task is to classify the last sentence of 'InputText' based on the provided examples. Each example shows a sentence, inverse similarity to InputText (the lower the better), and its corresponding classification label. Use these examples to determine the correct classification for the given text. Return only the classification label starting with 'T'.\n" + xs
    
    #####
    
    #print(xs)
    prompt = xs#prompt#"Write a poem about a cat chasing a butterfly."
    result = llm.invoke(prompt)
    predicted_label = result.content

    predicted_labels.append(predicted_label)
    print("actual:", row["Technique ID"], ", predicted:", predicted_label)
    
    
    #print(distances)
    #print("----")
   
    [scores_list.append(score) for score in distances.flatten()]

In [ ]:
cleaned_labels = [label.replace('Classification:', '') for label in predicted_labels]

# Example vectors containing strings
y_true = test["Technique ID"].values#["cat", "dog", "fish", "cat", "dog"]
#y_true = train_data.Key
y_pred = cleaned_labels#.values#["cat", "dog", "cat", "cat", "dog"]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_true, y_pred, average='weighted')
print(f"Recall: {recall}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1}")

In [ ]:
#scores_list

In [ ]:
from statistics import mean
print(max(scores_list))
print(mean(scores_list))
print(min(scores_list))

In [ ]:
len(retrieved_docs)